### Week 4: Setting Up and using Ollama for a Locally Hosted LLM


##### 1. Install Required Dependencies

(we've used these before, but just to make sure): 

- Python 3.8+
- Jupyter Notebook
- Ollama (instructions below)

In [1]:
# Install required Python libraries
!pip install ollama requests

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


##### 2. Setup Ollama

Ollama is a tool for running LLMs locally. Follow these steps to set it up:

1. Download Ollama:

* Visit the Ollama website and download the latest release for your operating system: https://ollama.com/download

* Alternatively, use the command line to install it if you are using linux:

In [ ]:
curl -fsSL https://ollama.com/install.sh | sh

2. Run Ollama:

* Start the Ollama server locally. 

In [ ]:
ollama serve # make sure you are running this in a terminal! might take a second to load

In [8]:
ollama serve # starts the server, which will handle requests to the LLM

SyntaxError: invalid syntax (2990350625.py, line 1)

3. Download a Model:

* Ollama supports various LLMs (e.g., LLaMA, GPT-J). For today, we chose to use llama3.1 since it is good for a personal assistant application. Gemma 2 9b is also another good model for this use case.

Download the llam3.1 model:

In [9]:
ollama run llama3.1:8b-instruct-q6_K

SyntaxError: invalid decimal literal (3062653468.py, line 1)

##### 3. Verify Ollama Setup
Ensure Ollama is running and accessible.



In [ ]:
import requests

# Test the Ollama API
response = requests.post("http://localhost:8080/api/generate", json={"model": "llama3.1", "prompt": "Hello, world!"})
print(response.json())

##### 4. Implementing it into JARVIS
Now that Ollama is set up, let's create a simple JARVIS-like assistant.

1. Define the Assistant Logic:

* Use Ollama's API to generate responses based on user input.

In [ ]:
def jarvis(prompt):
    # Send the prompt to Ollama
    response = requests.post(
        "http://localhost:8080/api/generate",
        json={"model": "llama2", "prompt": prompt}
    )
    if response.status_code == 200:
        return response.json()["response"]
    else:
        return "Error: Unable to generate a response."

# Test the JARVIS function
print(jarvis("What is the weather like today?"))

2. Add Voice Input/Output:

We used the speech recognition and text-to-speech libraries previously to enable voice interation. We'll be integrating them again to use them with our model. Feel free to look back on that code, and reuse it here. 

* Use libraries like speech_recognition and pyttsx3 to enable voice interaction.

In [ ]:
import speech_recognition as sr
import pyttsx3

recognizer = sr.Recognizer()
engine = pyttsx3.init()

def listen():
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print(f"You said: {text}")
            return text
        except sr.UnknownValueError:
            return "Sorry, I didn't catch that!"

def speak(text):
    print(f"JARVIS: {text}")
    engine.say(text)
    engine.runAndWait()

# test voice interaction
user_input = listen()
response = jarvis(user_input)
speak(response)

##### 5. Create a Flask Web Interface (If extra time, otherwise we will go more in depth in the next few weeks)
To make JARVIS accessible via a web interface, use Flask. You'll need to run the following first if you don't have flask:

In [ ]:
!pip install flask

In [ ]:
from flask import Flask, request, jsonify

# create a Flask app instance
app = Flask(__name__)

@app.route("/jarvis", methods=["POST"]) # define a route for the JARVIS endpoint

def jarvis_endpoint():
    # get JSON data from the request
    data = request.json

    # extract the "prompt" from the JSON data (default to empty string if not provided)
    prompt = data.get("prompt", "")

    # call the JARVIS function with the prompt
    response = jarvis(prompt)

    # return the response as JSON again
    return jsonify({"response": response})

# lastly, run the Flask app
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)